<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/Transformer/5-4-Gemma-SQL-Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets huggingface_hub safetensors

In [6]:
import logging


def setup_logging():
    # Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.WARNING,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[logging.StreamHandler()],
    )

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="google/gemma-2-2b-it")
pipe(messages)

In [3]:
# Example user input
user_query = "Who is the highest paid NBA player?"

# Example table structure
table_structure = {
    "columns": [
        "Team", "NAME", "Jersey", "POS", "AGE", "HT", "WT", "COLLEGE", "SALARY"
    ]
}

# Placeholder function to simulate gemma-2b0it model API call
def gemma_model_generate_sql(query, table_structure):
    # SQL query generated based on the table structure and user query
    if "highest paid" in query:
        return "SELECT NAME, SALARY FROM players ORDER BY CAST(SALARY AS DECIMAL) DESC LIMIT 1;"
    return ""

# Generate the SQL query using the model
sql_query = gemma_model_generate_sql(user_query, table_structure)

print("Generated SQL Query: ", sql_query)

Generated SQL Query:  SELECT NAME, SALARY FROM players ORDER BY CAST(SALARY AS DECIMAL) DESC LIMIT 1;


In [4]:
!wget https://github.com/meta-llama/llama-recipes/raw/main/recipes/3p_integrations/lamini/text2sql_memory_tuning/nba_roster.db

--2024-10-17 06:30:24--  https://github.com/meta-llama/llama-recipes/raw/main/recipes/3p_integrations/lamini/text2sql_memory_tuning/nba_roster.db
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/meta-llama/llama-recipes/main/recipes/3p_integrations/lamini/text2sql_memory_tuning/nba_roster.db [following]
--2024-10-17 06:30:25--  https://raw.githubusercontent.com/meta-llama/llama-recipes/main/recipes/3p_integrations/lamini/text2sql_memory_tuning/nba_roster.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57344 (56K) [application/octet-stream]
Saving to: ‘nba_roster.db’

nba_roster.db       100%[=

In [10]:
result="""SELECT NAME, SALARY FROM players ORDER BY CAST(SALARY AS DECIMAL) DESC LIMIT 1;"""

In [15]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

# Create a connection to the SQLite database. If the file doesn't exist it will be created.
conn = sqlite3.connect('nba_roster.db')

# Create a cursor object to execute SQL commands.
cursor = conn.cursor()

# Instead of creating a new table, get the existing table name from the database.
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
existing_tables = cursor.fetchall()
# Assuming there's only one table in the database, extract its name.
# Modify this if you have multiple tables and need to select a specific one.
table_name = existing_tables[0][0]

# Update the SQL query to use the correct table name.
result = f"SELECT NAME, SALARY FROM {table_name} ORDER BY CAST(SALARY AS DECIMAL) DESC LIMIT 1;"

# Commit the changes to the database.
conn.commit()

# Now, proceed with your data loading or other operations.
engine = create_engine('sqlite:///nba_roster.db')
df = pd.read_sql(result, con=engine)

In [16]:
df

,NAME,SALARY
0,Saddiq Bey,"$4,556,983"
